In [ ]:
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
!pip install gensim
from gensim.parsing.preprocessing import remove_stopwords
!pip install tabulate
from tabulate import tabulate
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alice\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words("english"))

cleaner = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')


In [ ]:
# functions
def extract(pdf_file:str):
    file_read = PdfReader(pdf_file)
    pdf_text=""
    for page in file_read.pages:
        content = page.extract_text()
        pdf_text += content
    return pdf_text 

def cleanhtml(raw_html):
  cleantext = re.sub(cleaner, '', raw_html)
  return cleantext

def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def remove_whitespace(text):
    return  " ".join(text.split())

def cut_in_sliding_windowns(text):
    separations=[]
    text_to_words=word_tokenize(text)
    n=len(text_to_words)
    i=0
    while i<n:
        separations.append(i)
        i=i+200
    list_section=[]
    for i in separations:
        list_section.append(text_to_words[i:i+250])
    return list_section

def count_esg(text):
    count_ecolo = 0
    count_social = 0 
    count_governance = 0 
    for word in word_tokenize(text):
        count_neg += 1 if word.upper() in neg else 0
        count_pos += 1 if word.upper() in pos else 0
    return (count_pos-count_neg)/(count_pos+count_neg)

def select_esg_sentences(datafr):
    data_esg=datafr.copy()
    data_esg.drop(data_esg[data_esg['label'] == 'None'].index, inplace = True)
    return data_esg 
#Bert doesn't work for sentences of more than 512 words; I just drop them
#the percentage of such 'too long' sentences is the last value of the returned array

def bert_phrase(list):
    finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
    tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
    nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
    n = len(list)
    sent=[]
    label=[]
    score=[]
    if n>0: 
        for sentence in list:
            if len(sentence)<512:
                result = nlp(sentence)
                sent.append(sentence)
                label.append(result[0]['label'])
                score.append(result[0]['score'])
                n=n-1
            else:
                n=n-1
    df_phrases = pd.DataFrame({'sentence':sent, 'label': label, 'score':score}, columns=['sentence', 'label', 'score'])
    return df_phrases 
#Bert doesn't work for sentences of more than 512 words; I just drop them
#the percentage of such 'too long' sentences is the last value of the returned array

In [ ]:
x = extract('10-KFord2022.pdf')
x = cleanhtml(x)
x = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", x)
x= x.lower()
x= remove_numbers(x)
x = x.split(".") 
for i in range (len(x)):
    x[i] = re.sub(r'[^\w]', ' ', x[i])
for phrase in x:
    if (len(phrase)<15):
        x.remove(phrase)

In [ ]:
print(x[0])

   united states securities and exchange commission washington  dc  form  k   annual report pursuant to section  or  d  of the securities exchange act of  for the fiscal year ended december    or   transition report pursuant to section  or  d  of the securities exchange act of  for the transition period from  __________ to __________ commission file number     ford motor company  exact name of registrant as specified in its charter  delaware    state of incorporation   i


In [ ]:
bert_phrases = bert_phrase(x)

In [ ]:
bert_phrases

,sentence,label,score
0,united states securities and exchange commi...,None,0.991014
1,employer identification no,None,0.628254
2,one american road dearborn michigan add...,None,0.981163
3,per share f new york stock exchange,None,0.983001
4,notes due june fprb new york stock exchange,None,0.994293
...,...,...,...
3880,for the year ended includes million rel...,None,0.996267
3881,b accounts receivable deemed to be uncollect...,None,0.995085
3882,c net change in inventory allowances includ...,None,0.995496
3883,d change in valuation allowance on deferre...,None,0.995705


In [ ]:
bert_phrases.to_csv('Ford2022_Select_ESG_Sentences_via_BERT.csv')

In [ ]:
df = pd.read_csv('Ford2022_Select_ESG_Sentences_via_BERT.csv')
df=df.drop('Unnamed: 0',axis=1)

In [ ]:
esg_sentences=select_esg_sentences(df)
esg_sentences.to_csv('Ford2022_onlyEsgSentences_Select_ESG_Sentences_via_BERT.csv')